In [1]:
from sqlmodel import SQLModel, Field, Relationship, ForeignKey
from typing import List
from beeprint import pp


In [2]:
class Author(SQLModel, table=True):
    id: int = Field(primary_key=True)
    name: str
    email: str

class Book(SQLModel, table=True):
    id: int = Field(primary_key=True)
    title: str
    author_id: int = Field(default=None,  foreign_key="author.id", description="auyjor id")



In [3]:
import json


def get_model_structure(model_class: type[SQLModel]) -> dict:
    """Get the structure of a given SQLModel as a dictionary."""
    structure = {
        "fields": {},
        "relationships": {}
    }


    for field in model_class.__table__.columns:
        # Add each field to the "fields" dictionary
        pp(field)

        structure["fields"][field.name] = {
            "type": str(field.type),
            "default": getattr(field, "default", None),
            "primary_key": field.primary_key,
            "unique": field.unique,
            "index": field.index,
            "foreign_keys": []
        }

    # If model_class has any relationships
    if hasattr(model_class, "__mapper__"):
        for rel in model_class.__mapper__.relationships:
            structure["relationships"][rel.key] = {
                "model": rel.target.name,
                "direction": rel.direction.name
            }

    with open("model_structure.json", 'w') as f:
        json.dump(structure, f, indent=4)

    print(structure)
    return structure


    # for field in model_class.__table__.columns:
    #     # pp(field)
    #     structure["fields"][field.name] = {"type": field.type, "default": getattr(field, "default", None)}

    # print(structure)
    # return structure


In [4]:
def create_prompt(model_class: type[SQLModel]) -> str:
    """Create a prompt for Llama 3.1 to generate synthetic data."""
    structure = get_model_structure(model_class)
    fields = ", ".join(f"{field} (e.g., '{structure['fields'][field]['default']}')" for field in structure["fields"])
    return f"Generate a sample of the following data: {', '.join(structure['fields'].keys())}."


In [5]:
prompt = create_prompt(Book)
# from beeprint import pp
# print(prompt)


Column('id', Integer(), table=<book>, primary_key=True, nullable=False)
Column('title', AutoString(), table=<book>, nullable=False)
Column('author_id', Integer(), ForeignKey('author.id'), table=<book>, nullable=False)
{'fields': {'id': {'type': 'INTEGER', 'default': None, 'primary_key': True, 'unique': False, 'index': False, 'foreign_keys': []}, 'title': {'type': 'VARCHAR', 'default': None, 'primary_key': False, 'unique': False, 'index': False, 'foreign_keys': []}, 'author_id': {'type': 'INTEGER', 'default': None, 'primary_key': False, 'unique': False, 'index': False, 'foreign_keys': []}}, 'relationships': {}}


In [6]:
# Use Llama 3.1 to generate synthetic data based on the prompt and add it to your model using SQLModel's add() method
import ollama


In [7]:
import requests
import json
import ollama

data = {
            "prompt": prompt,
            "model": 'llama3.1:8b',
            "format": "json",
            "stream": False,
            "options": {"temperature": 100},
        }

response = requests.post("http://localhost:11434/api/generate", json=data, stream=False)
json_data = json.loads(response.text)
print(f"Raw AI response:\n {json_data}")   
    

Raw AI response:
 {'model': 'llama3.1:8b', 'created_at': '2024-10-16T00:01:14.19660695Z', 'response': '{ \n"id": 1,\n"title": "Harry Potter and the Philosopher\'s Stone",\n"author_id": 101 \n\n}\n\n \n ', 'done': True, 'done_reason': 'stop', 'context': [128006, 882, 128007, 271, 32215, 264, 6205, 315, 279, 2768, 828, 25, 887, 11, 2316, 11, 3229, 851, 13, 128009, 128006, 78191, 128007, 271, 90, 720, 29800, 794, 220, 16, 345, 83827, 794, 330, 42398, 30427, 323, 279, 7302, 46459, 596, 14637, 761, 1, 3170, 851, 794, 220, 4645, 4815, 633, 720, 220], 'total_duration': 5362273516, 'load_duration': 4586598549, 'prompt_eval_count': 25, 'prompt_eval_duration': 94038000, 'eval_count': 30, 'eval_duration': 637800000}


In [8]:
book_instance = Book(**json_data[0])  # Add the generated data to a new instance of the Book model


KeyError: 0